In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from scipy import stats

In [2]:
dataset_path = '/shareds/IEMOCAP/IEMOCAP_video'
data = pd.read_csv(os.path.join(dataset_path, 'markup.csv'))
data.head()

,session,fn,idx,sex,emotion,text
0,1,Ses01F_impro07,0,M,exc,Did you get the letter?
1,1,Ses01F_impro07,1,F,exc,"Yes. There's a big envelope it says, you're i..."
2,1,Ses01F_impro07,2,M,exc,Yeah. That is so awesome.
3,1,Ses01F_impro07,4,M,exc,Oh my God. What are you going to do? [LAUGHTER]
4,1,Ses01F_impro07,5,F,exc,So I have to move back to the ghetto but...I k...


In [3]:
au_info = pd.read_csv('au_info.csv')
au_info.head()

,AU,FACS Name,Muscles,FACS Category,Related Expression,Models,Notes
0,AU01,Inner Brow Raiser,Frontalis (medial),main,"sadness, surprise, fear","svm, xgb, Py-Feat viz",NaN
1,AU02,Outer Brow Raiser,Frontalis (lateral),main,"surprise, fear","svm, xgb, Py-Feat viz",NaN
2,AU03,Inner corner Brow Tightener,"Procerus, Depressor Supercilii, Corrugator Sup...",extended (Baby FACS),NaN,none,Only in babies! Analogue of AU4 in adults
3,AU04,Brow Lowerer,"Procerus, Depressor Supercilii, Corrugator Sup...",main,"sadness, fear, anger","svm, xgb, Py-Feat viz",NaN
4,AU05,Upper Lid Raiser,"Levator Palpebrae Superioris, Superior Tarsal ...",main,"surprise, fear, anger","svm, xgb, Py-Feat viz",NaN


In [4]:
au2text = {row['AU']: row["FACS Name"] for i, row in au_info.iterrows()}

In [5]:
emo_au = { # https://en.wikipedia.org/wiki/Facial_Action_Coding_System
    "happiness": ["AU06", "AU12"],
    "sadness": ["AU01", "AU04", "AU15"],
    "surprise": [ "AU01", "AU02", "AU05", "AU26"],
    "fear": ["AU01", "AU02", "AU04", "AU05", "AU07", "AU20", "AU26"],
    "anger": [ "AU04", "AU05", "AU07", "AU23"],
    "disgust": ["AU09", "AU15", "AU17"]
}

In [6]:
from feat import Detector

detector = Detector(device='cuda')
detector

Detector(face_model=img2pose, landmark_model=mobilefacenet, au_model=xgb, emotion_model=resmasknet, facepose_model=img2pose, identity_model=facenet)

In [7]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [8]:
emo_labels = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']

In [ ]:
au_descs = []

for i, row in tqdm(data.iterrows(), total=data.shape[0]):
    video_path = os.path.join(dataset_path, f"Session{row['session']}", row['fn'], f"{row['idx']}.avi")
    video_prediction = detector.detect(
        video_path,
        data_type='video', 
        skip_frames=24,
        face_detection_threshold=0.95, 
        progress_bar=False,
        batch_size=8
    )
    spk_preds = video_prediction[video_prediction['Identity'] == 'Person_0']
    pseudo_label = stats.mode(np.array(spk_preds.emotions[emo_labels[:-1]]).argmax(axis=1))[0]
    pseudo_emo = emo_labels[pseudo_label]
    aus = emo_au[pseudo_emo]
    desc = ""
    for au in aus:
        desc += f"{au2text[au]}, "
    desc = desc[:-2]
    au_descs.append(desc)

  0%|          | 0/1 [00:00<?, ?it/s]


In [ ]:
mod_data['aus'] = au_descs
mod_data.head()

In [ ]:
mod_data.to_csv('../data/IEMOCAP/modified_au_data.csv')